In [1]:
"""
Created on Mar 06 2022
@author: Jerónimo Arenas García

Temporary routine for generation of JSON files
simulating the creation of files required for the
BI tool

If other datasets want to be ingested, we need to
1) Change the SELECT statement accordingly
2) Change the fields as necessary
3) Change the name of the folder where the JSON will be save
"""

import TMinferencer
import io
from langdetect import detect
from pyspark.sql import SparkSession
from pyspark.sql.functions import array_contains, concat_ws, col, udf
from pyspark.sql.types import StringType

In [2]:
sc.addPyFile("TMinferencer.py")

In [15]:
# Read full table of CORDIS projects
projects = spark.sql("SELECT * FROM parquet.`/export/ml4ds/IntelComp/Datalake/CORDIS/20220221/new_parquet/projects.parquet`")

22/05/26 00:25:39 WARN metastore.ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [16]:
def TMinference(rawt):
    thetas = TMinferencer.main(io.StringIO(rawt.replace("\n", " ")))
    return " ".join("t"+str(el[0])+"|"+str(round(1000*el[1])) for el in thetas[0].items())
udf_TMinference = udf(lambda x:TMinference(x), StringType() )

projects = projects.withColumn("TM40",udf_TMinference(col("title")))
projects.show(n=3, vertical=True)

-RECORD 0----------------------------------
 projectID          | 115843               
 acronym            | EbolaMoDRAD          
 status             | CLOSED               
 title              | Ebola Virus: Mode... 
 startDate          | 2015-02-01 00:00:00  
 endDate            | 2018-01-31 00:00:00  
 totalCost          | 4300935.0            
 ecMaxContribution  | 4300935.0            
 ecSignatureDate    | 2015-01-21 00:00:00  
 frameworkProgramme | H2020                
 masterCall         | H2020-JTI-IMI2-20... 
 subCall            | H2020-JTI-IMI2-20... 
 fundingScheme      | IMI2-RIA             
 nature             | null                 
 objective          | The current Ebola... 
 contentUpdateDate  | 2021-12-07 23:57:00  
 rcn                | 209451               
 grantDoi           | null                 
 topic              | IMI2-2014-02-05      
 topic_title        | Rapid diagnostic ... 
 countryContr       | BE|318125.0 DE|17... 
 orgContr           | 961534423|

In [17]:
projects = projects.drop("status").drop("nature").drop("objective").drop("contentUpdateDate").drop("rcn").drop("grantDoi")
projects.show(n=3, vertical=True)

-RECORD 0----------------------------------
 projectID          | 115843               
 acronym            | EbolaMoDRAD          
 title              | Ebola Virus: Mode... 
 startDate          | 2015-02-01 00:00:00  
 endDate            | 2018-01-31 00:00:00  
 totalCost          | 4300935.0            
 ecMaxContribution  | 4300935.0            
 ecSignatureDate    | 2015-01-21 00:00:00  
 frameworkProgramme | H2020                
 masterCall         | H2020-JTI-IMI2-20... 
 subCall            | H2020-JTI-IMI2-20... 
 fundingScheme      | IMI2-RIA             
 topic              | IMI2-2014-02-05      
 topic_title        | Rapid diagnostic ... 
 countryContr       | BE|318125.0 DE|17... 
 orgContr           | 961534423|251250.... 
 coordinatorCountry | SE                   
 coordinatorOrg     | 9.49789954E8         
 euroSciVocCode     | [155, 56306972, 3... 
 publicationID      | [115843_202840_PU... 
 patentID           | null                 
 TM40               | t7|998 t31

In [18]:
#Save corpus
projects.write.json(f"file:///export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/Cordis.json")

In [20]:
# Read CORDIS publications table
pubs = spark.sql("SELECT * FROM parquet.`/export/ml4ds/IntelComp/Datalake/CORDIS/20220221/new_parquet/publications.parquet`")
pubs.show(n=3, vertical=True)

22/05/26 00:32:31 WARN metastore.ObjectStore: Failed to get database parquet, returning NoSuchObjectException


AnalysisException: Unable to infer schema for Parquet. It must be specified manually.; line 1 pos 14

In [3]:
# Read subtable from parquet file
S2papers = spark.sql("SELECT id, year, venue, title, paperAbstract FROM parquet.`/export/ml4ds/IntelComp/Datalake/SemanticScholar/20220201/papers.parquet` where array_contains(fieldsOfStudy, 'Computer Science')")

22/03/06 22:03:50 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/03/06 22:03:50 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/03/06 22:03:52 WARN metastore.ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/03/06 22:03:52 WARN metastore.ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore jarenas@192.168.148.225
22/03/06 22:03:52 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
22/03/06 22:03:52 WARN metastore.ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [5]:
def TMinference(rawt):
    thetas = TMinferencer.main(io.StringIO(rawt.replace("\n", " ")))
    return " ".join("t"+str(el[0])+"|"+str(round(1000*el[1])) for el in thetas[0].items())
udf_TMinference = udf(lambda x:TMinference(x), StringType() )

S2papers = S2papers.withColumn("TM40",udf_TMinference(col("rawtext")))
S2papers = S2papers.drop("rawtext")


In [6]:
#Save corpus
S2papers.repartition(100).write.json(f"file:///export/usuarios_ml4ds/jarenas/github/IntelComp/ITMT/topicmodeler/S2CS.json")

In [9]:
projects.show(n=10)

+---------+-----------------+--------------------+-------------------+-------------------+-----------+-----------------+-------------------+------------------+--------------------+--------------------+-------------+-------------------+--------------------+--------------------+--------------------+------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|projectID|          acronym|               title|          startDate|            endDate|  totalCost|ecMaxContribution|    ecSignatureDate|frameworkProgramme|          masterCall|             subCall|fundingScheme|              topic|         topic_title|        countryContr|            orgContr|coordinatorCountry|coordinatorOrg|      euroSciVocCode|       publicationID|            patentID|                TM40|
+---------+-----------------+--------------------+-------------------+-------------------+-----------+-----------------+-------------------+------------------+-------